# MANUAL DATA STANDARDIZATION

----------------------------
### GIST OF CHANGES DONE IN THIS NOTEBOOK
----------------------------
-   The primary appproach for standardizing code descriptions was to replace some commonly occuring verbs and terms across multiple rows with desired standardization
- <b>STANDARDIZATION APPROACH</b>  
  -   Standardize the code to contain standard column names like COL_A, COL B etc and standard literals like LIT_STR_1, LIT_INT_1, LIT_DEC_1 etc for various types of literals using regex
  -   Incorporate the standardized columns and literals above into the code descriptions to establish a direct relation between the code and descriptions via these standards
  -   Standardization are applied primarily to functions using up to three standardized arguments. Some complicated code and description were not standardized in the interest of time and due to the complicated nature of the code.
  -   A very small set of code descriptions which were found to be incoherent were adjusted to describe the code briefly
-   <b>NOTE</b>: Standardization done below will be appended to the original dataset hence retaining the original code and descriptions
----------------------------
<br>
<br>


In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# IMPORT NECESSARY PACKAGES
import io
import re
import pandas as pd
from tqdm import tqdm
from google.colab import drive, files

import warnings
warnings.simplefilter(action='ignore')


In [3]:
# DISPLAY FUNCTIONS
def fn_display_header(msg):
  print('-' * 80)
  print(' ' * 10, msg)
  print('-' * 80)

def fn_display_message(msg):
  print(msg)

In [6]:
# READ WEB SCRAPED DATA INTO DATAFRAME AND DISPLAY DETAILS

df_raw = pd.read_csv('ETL_P2_model_input_data_v1.csv').drop('Unnamed: 0', axis=1)
df_raw['code_description'] = df_raw['code_description'].str.replace(r'\s+', ' ', regex=True)
df_raw['code_snippet'] = df_raw['code_snippet'].str.replace('save(my_dir)AnalysisException:', 'save(my_dir)')
df_raw['code_snippet'] = df_raw['code_snippet'].str.replace('my_dir', 'MY_DIR')

fn_display_header('Display Dataframe Metadata')
df_raw.info()

fn_display_header('Display initial Web Scraped Dataframe')
#df_raw.style.set_properties(**{'text-align': 'left'})
df_raw.head()


--------------------------------------------------------------------------------
           Display Dataframe Metadata
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   code_description  694 non-null    object
 1   code_snippet      694 non-null    object
 2   import_line       62 non-null     object
 3   Category          694 non-null    object
 4   function          694 non-null    object
dtypes: object(5)
memory usage: 27.2+ KB
--------------------------------------------------------------------------------
           Display initial Web Scraped Dataframe
--------------------------------------------------------------------------------


,code_description,code_snippet,import_line,Category,function
0,Pyspark code to Write a DataFrame into a CSV file,"df.write.mode(""overwrite"").format(""csv"").save(...",NaN,Input/Output,pyspark.sql.DataFrameReader.csv
1,Pyspark code which Loads a CSV file and return...,"df.write.mode(""overwrite"").format(""csv"").save(...",NaN,Input/Output,pyspark.sql.DataFrameReader.csv
2,Pyspark code to Read the CSV file as a DataFra...,"df = spark.read.csv(MY_DIR, schema=df.schema, ...",NaN,Input/Output,pyspark.sql.DataFrameReader.csv
3,Pyspark code which Loads a CSV file and return...,"df = spark.read.csv(MY_DIR, schema=df.schema, ...",NaN,Input/Output,pyspark.sql.DataFrameReader.csv
4,Pyspark code to Write a DataFrame into a JSON ...,"df.write.mode(""overwrite"").format(""json"").save...",NaN,Input/Output,pyspark.sql.DataFrameReader.format


<br>
<br>
<br>

----
# <b>CODE STANDARDIZATION: DATAFRAME NAMES</b>
----
<br>

In [7]:
# STANDARDIZATION: DATAFRAME NAMES

df_standardize_df_names = df_raw.copy()

# Function to standardize dataframe names as MY_DF
def fn_standardize_df_names(code, function):
  standard_df_name = 'MY_DF'
  fn_main = function.split('.')[-1].strip()

  code = code.replace('df \ .', 'df.').replace('rodf.', f'{standard_df_name}.')

  replace_str_list1 = re.findall(r'(df[a-zA-Z0-9]*)\.', code)
  replace_str_list1 = [x + '.' for x in replace_str_list1]
  replace_str_list1 = list(set(replace_str_list1))

  replace_str_list2 = re.findall(r'(df[a-zA-Z0-9]*)\)\)', code)
  replace_str_list2 = [x + '.' for x in replace_str_list2]
  replace_str_list2 = list(set(replace_str_list2))

  replace_str_list_final = replace_str_list1 + replace_str_list2
  for rpl in replace_str_list_final:
    code = code.replace(rpl, f'{standard_df_name}.')

  special_str_list = ['df = ', 'df["']
  for item in special_str_list:
    if item in code:
      code = code.replace('df', f'{standard_df_name}')

  if '_union_' in fn_main and 'df2' in code:
    code = code.replace('df1 = ', '').replace('df2 = ', '').replace('df3 = ', '').replace('df4 = ', '').replace('df5 = ', '')
    code = code.replace(f'{standard_df_name}.union(df2)', f'{standard_df_name}1.union(MY_DF2)')
  if '_sketch_' in fn_main:
    code = code.replace('df2 = ', '').replace('df3 = ', '')

  result = code
  return result


# Standardize Dataframe names to MY_DF
df_standardize_df_names['code_snippet_1'] = df_raw.apply(lambda x: fn_standardize_df_names(x['code_snippet'], x['function']), axis=1)

# UNCOMMENT TO DEBUG
df_standardize_df_names[
    #(df_standardize_df_names['function'].str.contains('pyspark.sql.DataFrameReader.option'))
    (df_standardize_df_names['code_snippet_1'].str.contains('MY_DF.'))
    #& (~df_standardize_df_names['code_snippet_1'].str.startswith('MY_DF.'))
#].info()
#].count()
].style.set_properties(**{'text-align': 'left'})


#df_standardize_df_names.style.set_properties(**{'text-align': 'left'})
df_standardize_df_names.head().style.set_properties(**{'text-align': 'left'})


,code_description,code_snippet,import_line,Category,function,code_snippet_1
0,Pyspark code to Write a DataFrame into a CSV file,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)"
1,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)"
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'Hyukjin Kwon'.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""Hyukjin Kwon"")"
3,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""Hyukjin Kwon"")"
4,Pyspark code to Write a DataFrame into a JSON file,"df.write.mode(""overwrite"").format(""json"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.format,"MY_DF.write.mode(""overwrite"").format(""json"").save(MY_DIR)"


<br>
<br>
<br>

----
# <b>CODE STANDARDIZATION: LITERALS</b>
----
<br>

In [8]:
# STANDARDIZATION: LITERALS are replaced with a standard prefix LIT_INT, LIT_DEC, LIT_STR.
#                  ALIASES are replaced with MY_ALIAS_A, MY_ALIAS_B

# Function to standardize literals
def fn_standardize_literals(code, function):
  literal_prefix = 'LIT'
  fn_main = function.split('.')[-1].strip()

  lit_match = []
  int_common_idx = -1
  dec_common_idx = -1
  str_common_idx = -1
  # Match positive integers
  lit_match1 = re.findall(r'lit\((\d+?)\)', code)
  for idx, item in enumerate(lit_match1):
    int_common_idx = int_common_idx + 1
    code = code.replace(f'lit({item})', f'lit({literal_prefix}_INT_{int_common_idx})')

  # Match negative integers
  lit_match2 = re.findall(r'lit\(-(\d+?)\)', code)
  for idx, item in enumerate(lit_match2):
    int_common_idx = int_common_idx + 1
    code = code.replace(f'lit(-{item})', f'lit({literal_prefix}_INT_{int_common_idx})')

  # Match positive Decimals
  lit_match3 = re.findall(r'lit\((\d+\.\d+)\)', code)
  for idx, item in enumerate(lit_match3):
    dec_common_idx = dec_common_idx + 1
    code = code.replace(f'lit({item})', f'lit({literal_prefix}_DEC_{dec_common_idx})')

  # Match negative Decimals
  lit_match4 = re.findall(r'lit\(-(\d+\.\d+)\)', code)
  for idx, item in enumerate(lit_match4):
    dec_common_idx = dec_common_idx + 1
    code = code.replace(f'lit(-{item})', f'lit({literal_prefix}_DEC_{dec_common_idx})')

  # Match strings with patter lit("<str>")
  lit_match5 = re.findall(r'lit\("+(\w+?)"+\)', code)
  for idx, item in enumerate(lit_match5):
    str_common_idx = str_common_idx + 1
    code = code.replace(f'lit("{item}")', f'lit("{literal_prefix}_STR_{str_common_idx}")')

  # Match strings with patter lit('<str>')
  lit_match5 = re.findall(r"lit\('+(\w+?)'+\)", code)
  for idx, item in enumerate(lit_match5):
    str_common_idx = str_common_idx + 1
    code = code.replace(f"lit('{item}')", f'lit("{literal_prefix}_STR_{str_common_idx}")')

  #Handle alias(<>) as alias("MY_ALIAS")
  alias_match1 = re.findall(r"alias\('+(\w+?)'+\)", code)
  if len(alias_match1) == 1:
    code = code.replace(f"alias('{alias_match1[0]}')", f'alias(MY_ALIAS_A)')
  alias_match2 = re.findall(r'alias\("+(\w+?)"+\)', code)
  if len(alias_match2) == 1:
    code = code.replace(f'alias("{alias_match2[0]}")', f'alias(MY_ALIAS_A)')
  if 'alias(' in code:
    code = code.replace('alias("key", "value"', 'alias(MY_ALIAS_A, MY_ALIAS_B')

  if '"dotNET", "Java"' in code:
    code = code.replace('"dotNET", "Java"', f'"{literal_prefix}_STR_1", "{literal_prefix}_STR_2"')

  num_lit_match = re.findall(r'\(\d+?\)', code)
  if len(num_lit_match) == 1:
    code = code.replace(f'{num_lit_match[0]}', '(LIT_DEC_1)')

  # Special cases
  code = code.replace("('a', 1)", "('a', LIT_INT_1)")
  code = code.replace(", 2, 16", ", LIT_INT_1, LIT_INT_2")
  code = code.replace('MY_DF.dt, 1', 'MY_DF.COL_A, LIT_INT_1')
  code = code.replace('"dt", -1', '"dt", LIT_INT_1').replace("'dt', -1", "'dt', LIT_INT_1").replace("'dt', -2", "'dt', LIT_INT_1").replace('"dt", 1', '"dt", LIT_INT_1')
  code = code.replace('5 seconds', 'LIT_INT_1 seconds')
  code = code.replace('MY_DF.data, 1', 'MY_DF.data, LIT_INT_1').replace('MY_DF.data, 3', 'MY_DF.data, LIT_INT_1').replace('MY_DF.data, 5', 'MY_DF.data, LIT_INT_1').replace('MY_DF.data, -1', 'MY_DF.data, LIT_INT_1')
  code = code.replace('"data", 1', '"data", LIT_INT_1').replace('"data", -1', '"data", LIT_INT_1')
  code = code.replace('"c2", 2, -1', '"c2", LIT_INT_1, LIT_INT_2').replace('"c2", 1, 0', '"c2", LIT_INT_1, LIT_INT_2').replace('"c2", 1', '"c2", LIT_INT_1').replace('"c2", 2', '"c2", LIT_INT_1')
  code = code.replace("'a', 4", "'a', LIT_INT_1")
  code = code.replace("'l', 'r', 2", "'l', 'r', LIT_INT_1")
  code = code.replace('MY_DF.s, 1, 2', 'MY_DF.s, LIT_INT_1, LIT_INT_2')
  code = code.replace('MY_DF.s, 1', 'MY_DF.s, LIT_INT_1').replace('MY_DF.s, 6', 'MY_DF.s, LIT_INT_1').replace("MY_DF.s, '.', 2", "MY_DF.s, '.', LIT_INT_1").replace("MY_DF.s, '.', -3", "MY_DF.s, '.', LIT_INT_1")
  code = code.replace('"x", "y", 7, 0', '"x", "y", LIT_INT_1, LIT_INT_2').replace('"x", "y", 7, 2', '"x", "y", LIT_INT_1, LIT_INT_2').replace('"x", "y", 7', '"x", "y", LIT_INT_1')
  code = code.replace(', "rnlt", "123"', ', "rnlt", "LIT_STR_1"')
  code = code.replace('MY_DF.x, 2, 2', 'MY_DF.x, LIT_INT_1, LIT_INT_2')
  code = code.replace("MY_DF.s, 'b'", "MY_DF.COL_A, LIT_STR_1").replace('MY_DF.s, 3', 'MY_DF.s, LIT_INT_1')
  code = code.replace("'b', MY_DF.COL_A, 1", "'LIT_STR_1', MY_DF.COL_A, LIT_INT_1")
  code = code.replace('raise_error("My error message"', 'raise_error("MY_ERROR_MSG"')


  lit_match = lit_match1 + lit_match2 + lit_match3 + lit_match4 + lit_match5 + alias_match1 + alias_match2

  #if len(lit_match) > 0 and function == 'pyspark.sql.functions.sign':
  #  print(f'--DEBUG: lit_match1:{lit_match1} -- lit_match2:{lit_match2} -- lit_match3:{lit_match3} -- lit_match4:{lit_match4} -- ')

  result = [code, lit_match]
  return result

# Standardize Dataframe names to MY_DF
df_standardize_literals = df_standardize_df_names.copy()
#df_standardize_literals['literals'] = df_standardize_literals.apply(lambda x: fn_standardize_literals(x['code_snippet_1'], x['function'])[1], axis=1)  # ENABLE FOR DEBUGGING ONLY
df_standardize_literals['code_snippet_1'] = df_standardize_literals.apply(lambda x: fn_standardize_literals(x['code_snippet_1'], x['function'])[0], axis=1)

# UNCOMMENT TO DEBUG

df_standardize_literals[
    #(~df_standardize_literals['code_snippet_1'].str.contains('LIT_'))
    #(df_standardize_literals['code_snippet_1'].str.contains('alias\('))
    #(df_standardize_literals['code_snippet_1'].str.contains(r'"(\w+?)"'))
    #(df_standardize_literals['code_snippet'].str.contains(r'\(\d+?\)'))
    (df_standardize_literals['code_snippet'].str.contains(r'\d+?'))
    & (~df_standardize_literals['function'].str.contains(r'DataFrameReader'))
    & (~df_standardize_literals['function'].str.contains(r'DataFrameWriter'))
    #(df_standardize_literals['code_snippet'].str.contains(r'lit\("(\w+?)"\)'))
    #(df_standardize_literals['code_snippet'].str.contains(r"lit\('(\w+?)'\)"))
    #(df_standardize_literals['code_snippet_1'].str.contains('lit\('))
#].info()
#].count()
].style.set_properties(**{'text-align': 'left'})

#df_standardize_literals.style.set_properties(**{'text-align': 'left'})
df_standardize_literals.head().style.set_properties(**{'text-align': 'left'})


,code_description,code_snippet,import_line,Category,function,code_snippet_1
0,Pyspark code to Write a DataFrame into a CSV file,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)"
1,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)"
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'Hyukjin Kwon'.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""Hyukjin Kwon"")"
3,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""Hyukjin Kwon"")"
4,Pyspark code to Write a DataFrame into a JSON file,"df.write.mode(""overwrite"").format(""json"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.format,"MY_DF.write.mode(""overwrite"").format(""json"").save(MY_DIR)"


<br>
<br>
<br>

----
# <b>CODE STANDARDIZATION: COLUMN NAMES</b>
----
<br>

In [9]:
# STANDARDIZATION: Standardize random column names to COL_A, COL_B, COL_C, COL_D, COL_E

# Function to standardize column names
def fn_standardize_col_names(code, function, level):
  standard_df_name = 'MY_DF'
  ignore_prefix = code[:6]
  code = code[6:]
  fn_main = function.split('.')[-1].strip()

  #print(f'-- DEBUG --: ignore_prefix: {ignore_prefix} -- code: {code}')

  # Call function with level 3 argument to replace triple column list first
  if level == 3:

    # Replace columns in special cases frst
    spl_fn = ['first', 'last', 'max_by', 'median', 'min_by', 'mode', 'hex', 'sum', 'agg', 'explode_outer', 'posexplode_outer', 'grouping', 'applyInPandas', 'pivot', 'avg', 'min', 'max', 'sum']
    spl_cols = ['name',    'age', 'course',  'year', 'earnings', 'height',    'id', 'a_map', 'an_array',   "'b'",   'val', 'year', 'course', 'earnings', 'sales.year', 'sales.course', 'sales.earnings']
    spl_rplc = ['COL_A', 'COL_B',  'COL_C', 'COL_D',    'COL_E',  'COL_C', 'COL_A', 'COL_B',    'COL_C', 'COL_B', 'COL_B', 'COL_A', 'COL_B',    'COL_C',      'COL_A',        'COL_B',          'COL_C']
    if fn_main in spl_fn \
      and (
        'groupBy' in code or 'explode_outer' in code or 'grouping' in code or f"{fn_main}(" in code
      ):
      #if fn_main == 'first':
      #  print(f" --- code: {code}")
      for item, replacement in zip(spl_cols, spl_rplc):
        if 'MY_DF.' in code:
          code = code.replace(f'MY_DF.{item}', f'MY_DF.{replacement}')
        else:
          code = code.replace(f'"{item}"', f'"{replacement}"')

    code = code.replace('"COL_A", lead("c2"', '"next_value", lead("COL_B"')
    code = code.replace('unbase64(MY_DF.input), MY_DF.key, MY_DF.mode', 'unbase64(MY_DF.COL_A), MY_DF.COL_B, MY_DF.COL_C')
    code = code.replace("MY_DF.data, MY_DF.pos.cast('integer'), MY_DF.val", "MY_DF.COL_A, MY_DF.COL_B.cast('integer'), MY_DF.COL_C")
    code = code.replace('unbase64(MY_DF.input), MY_DF.key, MY_DF.mode, MY_DF.padding, MY_DF.aad', 'unbase64(MY_DF.COL_A), MY_DF.COL_B, MY_DF.COL_C, MY_DF.COL_D, MY_DF.COL_E')
    code = code.replace('unbase64(MY_DF.input), MY_DF.key, MY_DF.mode, MY_DF.padding', 'unbase64(MY_DF.COL_A), MY_DF.COL_B, MY_DF.COL_C, MY_DF.COL_D')
    code = code.replace('unhex(MY_DF.input), MY_DF.key', 'unhex(MY_DF.COL_A), MY_DF.COL_B')
    code = code.replace('MY_DF.id, ceil(MY_DF.v', 'MY_DF.COL_A, ceil(MY_DF.COL_B')
    code = code.replace('make_interval(MY_DF.a), MY_DF.b)', 'make_interval(MY_DF.COL_A), MY_DF.COL_B)')
    code = code.replace('MY_DF.d, make_interval(MY_DF.i)', 'MY_DF.COL_A, make_interval(MY_DF.COL_B)')
    code = code.replace("any_value('c1'), any_value('c2')", "any_value('COL_A'), any_value('COL_B')").replace("any_value('c1', True), any_value('c2'", "any_value('COL_A', True), any_value('COL_B'")
    code = code.replace('count_distinct(MY_DF.value, MY_DF.value)', 'count_distinct(MY_DF1.COL_A, MY_DF2.COL_A)')
    code = code.replace('"previos_value", lag("c2', '"COL_A", lag("COL_B')
    code = code.replace("unbase64(MY_DF.input), MY_DF.key, MY_DF.mode, MY_DF.padding, MY_DF.aad", "unbase64(MY_DF.COL_A), MY_DF.COL_B, MY_DF.COL_C, MY_DF.COL_D, MY_DF.COL_E")
    code = code.replace("unbase64(MY_DF.input), MY_DF.key, MY_DF.mode, MY_DF.padding", "unbase64(MY_DF.COL_A), MY_DF.COL_B, MY_DF.COL_C, MY_DF.COL_D")
    code = code.replace("hex('a'), hex('b')", "hex('COL_A'), hex('COL_B')")
    code = code.replace("sf.last_value('a'), sf.last_value('b')", "sf.last_value('COL_A'), sf.last_value(COL_B')").replace("sf.last_value('a', True), sf.last_value('b'", "sf.last_value('COL_A', True), sf.last_value('COL_B'")
    code = code.replace("'id', inline_outer(MY_DF.structlist)", "'COL_A', inline_outer(MY_DF.COL_B)")
    code = code.replace("(make_interval(MY_DF.i), make_interval(MY_DF.i)", "(make_interval(MY_DF.COL_A), make_interval(MY_DF.COL_B)")
    code = code.replace("MY_DF.dt, MY_DF.add.cast('integer')", "MY_DF.COL_A, MY_DF.COL_B.cast('integer')")


    date_columns = ['MY_DF.year', 'MY_DF.month', 'MY_DF.week', 'MY_DF.day', 'MY_DF.hour', 'MY_DF.min', 'MY_DF.sec']
    for item in date_columns:
      if item in code:
        code = code.replace(item, item.upper())
    code = code.replace('df.Y, df.M, df.D', 'df.YEAR, df.MONTH, df.DAY')

    triple_col_list = [x.replace('df.', f'{standard_df_name}.') for x in [
        'df.a, df.b, df.c', "df.vals1, df.vals2, df.vals3", "df.COL_A, df.COL_B, df.COL_C", "'C1', 'C2', 'C3'"
        ]]
    #print(f'-- DEBUG --:triple_col_list: {triple_col_list}')
    for item in triple_col_list:
      if 'df.' in item:
        code = code.replace(item, f'{standard_df_name}.COL_A, {standard_df_name}.COL_B, {standard_df_name}.COL_C')
      else:
        code = code.replace(item, f'"COL_A", "COL_B", "COL_C"')

  # Call function with level 2 argument to replace double column list after level 3 processing
  if level == 2:
    double_col_list = [x.replace('df.', f'{standard_df_name}.') for x in [
        'df.age, df.age', 'df.name, df.age', 'df.c1, df.c2', 'df.a, df.b, df.c', 'df.age, df.name', '"a", "b"', 'df.a, df.b', 'df.x, df.y',
        'df.k, df.v', 'unhex(df.input), df.key', 'df.s, df.d', 'df.data, "a"', 'df.ts, df.tz', 'df.d2, df.d1', 'df.birth, df.age', "df.c1, 'x'",
        "'age', 'age'", "'name', 'age'", "'age', 'name'", '"age", "height"', "'age', 'height'", '"x", "y"', '"y", "x"', '"map1", "map2"', 'df.date1, df.date2',
      'col("pattern"), col("replacement")', 'df.birth, df.age', 'df.d, df.i', "'c1', 'c2'", "'l', 'r'", '"c2", "c3"', "'C1', 'C2'", '"xs", "ys"', 'df.dt, df.sub'
        ]]
    #print(f'-- DEBUG --:double_col_list: {double_col_list}')
    for item in double_col_list:
      if 'df.' in item:
        code = code.replace(item, f'{standard_df_name}.COL_A, {standard_df_name}.COL_B')
      elif 'col(' in item:
          code = code.replace(item, 'col("COL_A"), col("COL_B")')
      else:
        code = code.replace(item, '"COL_A", "COL_B"')

  # Call function with level 2 argument to replace double column list after level 2 processing
  if level == 1:
    single_col_list1 = [x.replace('df', f'{standard_df_name}') for x in [
        'df["numbers"]', 'df.value', 'df.alphabets', 'df.input', 'df.string', 'df.name', 'df.data', 'df.id', #'df.schema',
        'df.a', 'df.b', 'df.c', 'df.c1', 'df.d', 'df.e', 'df.n', 'df.s', 'df.t', 'df.v', 'df.x', 'df.y'
        'df["id"]', 'col("id")', "col('c2')", 'col("value")', 'col("sketch")', 'col("numbers")', 'col("regexp")',
        'df.ts', 'df.mapfield', 'df.padding', 'df.aad', 'df.intlist', 'df.unix_time', 'df.dt', 'df.structlist'
        ]]
    #print(f'-- DEBUG --:single_col_list: {single_col_list}')
    for item in single_col_list1:
      for delim in [')', ',', ' ']:
        if f'{standard_df_name}[' in item:
          code = code.replace(f'{item}{delim}', f'{standard_df_name}["COL_A"]{delim}')
        elif f'{standard_df_name}.' in item:
          code = code.replace(f'{item}{delim}', f'{standard_df_name}.COL_A{delim}')
        elif 'col(' in item:
          code = code.replace(f'{item}{delim}', f'col("COL_A"){delim}')

    # Match single occurrences of columns manually
    single_col_list2 = [
        'id', 'a', 'c', 's', 'v', 'c1', 'c2', 'd2', 'cd', 'age', 'name', 'dt', 'date', 'values', 'value', 'val', 'data', 'desc_order', 'drank', 'pr', 'ntile', 'flag', 'distinct_cnt', 'sha2', 'str', 'cat', 'unix_time', 'ts',
        ]
    single_occ_columns = [f'"{x}"' for x in single_col_list2]
    single_occ_columns = single_occ_columns + [f"'{x}'" for x in single_col_list2]
    single_occ_ignore_fns = ['array_contains', 'array_position', 'sum', 'min', 'max', 'avg']
    for idx, item in enumerate(single_occ_columns):
      item_match = re.findall(item, code)
      if item in code and len(item_match) == 1 and fn_main not in ['array_contains', 'array_position', 'sum', 'min', 'max', 'avg']:
        code = code.replace(item, '"COL_A"')

  if fn_main == 'applyInPandas':
    code = code.replace('groupby("id"', 'groupby("COL_A"')
  if fn_main == 'get':
    code = code.replace('col("index")', 'col("COL_B")')
    code = code.replace('"index"', '"LIT_STR_0"')

  # Last minute fix for some replacements which did not work with generic code
  code = code.replace('age', 'COL_B') if fn_main == 'avg' else code
  code = code.replace('MY_DF.COL_A < MY_DF.COL_A', 'MY_DF.COL_A < MY_DF.COL_B').replace('MY_DF.COL_A > MY_DF.COL_A', 'MY_DF.COL_A > MY_DF.COL_B')   if fn_main == 'assert_true' else code
  code = code.replace('Hyukjin Kwon', 'LIT_STR_0')                                                                                                  if 'DataFrameWriter' in function or 'DataFrameReader' in function else code
  code = code.replace('MY_DF.Y, MY_DF.M, MY_DF.D', 'MY_DF.YEAR, MY_DF.MONTH, MY_DF.DAY')                                                            if fn_main == 'make_date' else code
  code = code.replace('substr("COL_A", "COL_B", "COL_A")', 'substr("COL_A", "COL_B", "COL_C")')                                                     if fn_main == 'substr' else code
  code = code.replace('"COL_A", "COL_B"', '"COL_B", "COL_C"')                                                                                       if fn_main == 'grouping_id' else code
  code = code.replace('col("COL_A"), col("COL_B")', 'col("COL_B"), col("COL_C")')                                                                   if fn_main == 'regexp_replace' else code
  code = code.replace('"rnlt"', '"LIT_STR_0"')                                                                                                      if fn_main == 'translate' else code
  code = code.replace('"COL_B", "COL_A"', '"COL_B", "COL_C"')                                                                                       if fn_main == 'printf' else code

  code = code.replace('MY_DF.COL_C, MY_DF.COL_A', 'MY_DF.COL_C, MY_DF.COL_D')                                                                       if fn_main == 'aes_decrypt' else code
  code = code.replace('MY_DF.COL_C, MY_DF.COL_D, MY_DF.COL_A', 'MY_DF.COL_C, MY_DF.COL_D, MY_DF.COL_E')                                             if fn_main == 'aes_decrypt' else code
  code = code.replace('MY_DF.COL_C, MY_DF.COL_A', 'MY_DF.COL_C, MY_DF.COL_D')                                                                       if fn_main == 'try_aes_decrypt' else code
  code = code.replace('MY_DF.COL_C, MY_DF.COL_D, MY_DF.COL_A', 'MY_DF.COL_C, MY_DF.COL_D, MY_DF.COL_E')                                             if fn_main == 'try_aes_decrypt' else code
  code = code.replace("'COL_B', 'height'", "'COL_B', 'COL_A'")                                             if fn_main == 'avg' else code

  result = ignore_prefix +  code
  return result

df_standardize_columns = df_standardize_literals.copy()

# Standardize Column names calling the function with level 3 first then 2 and then 1
df_standardize_columns['code_snippet_1'] = df_standardize_columns.apply(lambda x: fn_standardize_col_names(x['code_snippet_1'], x['function'], 3), axis=1)
df_standardize_columns['code_snippet_1'] = df_standardize_columns.apply(lambda x: fn_standardize_col_names(x['code_snippet_1'], x['function'], 2), axis=1)
df_standardize_columns['code_snippet_1'] = df_standardize_columns.apply(lambda x: fn_standardize_col_names(x['code_snippet_1'], x['function'], 1), axis=1)


# UNCOMMENT TO DEBUG
df_standardize_columns[
    #(df_standardize_columns['function'].str.contains('pyspark.sql.DataFrameReader.option'))
    #(df_standardize_columns['code_snippet_1'].str.contains('col\('))
    #(~df_standardize_columns['code_snippet_1'].str.contains('COL_A'))
    #(df_standardize_columns['code_snippet_1'].str.contains('COL_A'))
    #(~df_standardize_columns['function'].str.contains(r'DataFrameReader'))
    #& (~df_standardize_columns['function'].str.contains(r'DataFrameWriter'))
    (df_standardize_columns['code_snippet_1'].str.contains('group'))
    #& (~df_standardize_columns['code_snippet_1'].str.startswith('MY_DF.'))
#].info()
#].count()
].style.set_properties(**{'text-align': 'left'})

#df_standardize_columns.style.set_properties(**{'text-align': 'left'})
df_standardize_columns.head().style.set_properties(**{'text-align': 'left'})

,code_description,code_snippet,import_line,Category,function,code_snippet_1
0,Pyspark code to Write a DataFrame into a CSV file,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)"
1,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)"
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'Hyukjin Kwon'.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")"
3,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")"
4,Pyspark code to Write a DataFrame into a JSON file,"df.write.mode(""overwrite"").format(""json"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.format,"MY_DF.write.mode(""overwrite"").format(""json"").save(MY_DIR)"


<br>
<br>
<br>

----
# <b>CODE/COMMENT STANDARDIZATION: PASS 1</b>
----
<br>

In [10]:
# HANDLE COMMENTS PASS 1: Manipulate comments to incorporate standards introduced in code
#                         Handles Zero or single occurrences of COL or LITERAL  i.e. functions passing 1 argument

lit_search_id_dict = {
  'LIT_INT_0': 'integer',
  'LIT_INT_1': 'integer',
  'LIT_INT_2': 'integer',
  'LIT_INT_3': 'integer',
  'LIT_INT_4': 'integer',
  'LIT_INT_5': 'integer',
  'LIT_DEC_0': 'float or decimal',
  'LIT_DEC_1': 'float or decimal',
  'LIT_DEC_2': 'float or decimal',
  'LIT_DEC_3': 'float or decimal',
  'LIT_DEC_4': 'float or decimal',
  'LIT_DEC_5': 'float or decimal',
  'LIT_STR_0': 'string',
  'LIT_STR_1': 'string',
  'LIT_STR_2': 'string',
  'LIT_STR_3': 'string',
  'LIT_STR_4': 'string',
  'LIT_STR_5': 'string',
}

delim_dict = {
  'array_join': '","'
}

# Function to handle comments for referenes to MY_DIR and LITERALS
def fn_standardize_comments_1(comment, code, function, category):
  fn_main = function.split('.')[-1].strip()
  filter_str = '=NA='
  replaced_flag = 'N'

  # Handle some special cases for comments
  if not comment.strip().endswith('.'):
    comment = comment + '.'

  comment = comment.replace('Aggregate function', '').replace('Collection function', '').replace('Window function', '')
  comment = comment.replace('sensitiveinformation', 'sensitive information').replace('partitionsby', 'partitions by').replace('valuethen', 'value then').replace('thegiven', 'the given').replace('andis', 'and is')
  comment = comment.replace('NULLvalue', 'NULL value').replace('andkey', 'and key').replace('thearray', 'the array').replace('ofthe', 'of the').replace('givencolumn', 'given column').replace('andreturns', 'and returns')
  comment = comment.replace('0or', '0 or').replace('returnsa', 'returns a').replace('representationof', 'representation of').replace('fallafter', 'fall after').replace('dateformat', 'date format').replace('aswell', 'as well')
  comment = comment.replace('columnbased', 'column based').replace('withnull', 'with null').replace('valuein', 'value in').replace('nullvalue', 'null value').replace('anddefault', 'and default').replace('arraymust', 'array must')
  comment = comment.replace('functionreturns', 'function returns').replace('arraymust', 'array must').replace('allN th', 'all N-th').replace('resultby', 'result by').replace('countof', 'count of').replace('exceptionwith', 'exception with')
  comment = comment.replace('mapof', 'map of').replace('which This', 'which ').replace('numberand', 'number and').replace('Euler s', 'Eulers').replace('descendingorder', 'descending order').replace('functiontakes', 'function takes')
  comment = comment.replace('stringrepresenting', 'string representing').replace('defaultlocale', 'default locale').replace('timestampsto', 'timestamps to').replace('matchedin', 'matched in').replace('resultas', 'result as')
  comment = comment.replace('definedper', 'defined per').replace('urlencoded', 'url encoded')

  comment_orig = comment

  if fn_main in delim_dict:
    comment = comment.replace('delimiter', 'delimiter ' + delim_dict[fn_main])

  # Standardizes comments for DataFrameWriter and DataFrameReader functions
  if 'DataFrameWriter' in function or 'DataFrameReader' in function:
    comment = comment.replace('Hyukjin Kwon', 'LIT_STR_0')
    if 'MY_DIR' not in comment and 'MY_DIR' in code:
      comment = (comment + 'The data is written to file or directory MY_DIR')  if '.write' in code else comment
      comment = (comment + 'The data is read from file or directory MY_DIR')  if '.read' in code else comment
    filter_str = '1_MY_DIR'
    replaced_flag = 'Y'

  # search code for occurrences of standardized column names and literals
  arg_match = re.findall(r"(LIT_(INT|DEC|STR)_\d+|COL_[A-Z]|YEAR|MONTH|DAY|HOUR|MIN|SEC)", code)
  arg_match = list(dict.fromkeys([match[0] for match in arg_match])) # list(set([match[0] for match in arg_match]))

  arg_replace_list = []

  for item in arg_match:
    if 'COL_' in item or item in ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MIN', 'SEC']:
      arg_replace_list.append(f"column {item}")
    else:
      arg_replace_list.append(f"{lit_search_id_dict[item]} value {item}")

  df_suffix_str = " in dataframe MY_DF" if 'COL_' in code else ''

  if len(arg_match) == 1 and replaced_flag == 'N': # and 'DataFrameWriter' not in function and 'DataFrameReader' not in function:
    comment = comment.replace('given columns', 'column')
    filter_str = '1_INIT'

    if 'float' not in fn_main:
      comment = comment.replace('float', ' ')
    if fn_main == 'hash':
      comment = comment.replace('Pyspark code which Two or more columns', '.')

    # Handles explode functions and replaces with the appropriate standard
    if ('explode' in fn_main or fn_main == 'inline') and replaced_flag == 'N':
      comment = re.sub('array or map', f"array or map given by {arg_replace_list[0]}{df_suffix_str}", comment, 1).split('.')[0]
      comment = re.sub('array of structs', f"array of structs given by {arg_replace_list[0]}{df_suffix_str}", comment, 1).split('.')[0]
      filter_str = '1_fn_explode_1'
      replaced_flag = 'Y'
    # Checks for the occurrence of the word column and replaces with the appropriate standard
    elif ('column' in comment.lower() or ' col ' in comment or ' col.' in comment) and replaced_flag == 'N':
      comment = comment.replace('numeric columns for each group', 'numeric column for each group')
      comment = comment.replace(' col ', ' column ').replace('col.', 'column.')

      comment = re.sub('input column', f'input column {arg_replace_list[0]}{df_suffix_str}', comment, 1)
      comment = re.sub(r'column ', f"column {arg_replace_list[0]}{df_suffix_str} ", comment, 1)                   if f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub(r'column\.', f"column {arg_replace_list[0]}{df_suffix_str}.", comment, 1)                  if f"{arg_replace_list[0]}" not in comment else comment

      comment = comment.replace('column column', 'column')
      comment = comment.replace('column integer', 'integer').replace('column float', 'float').replace('column string', 'string')
      comment = comment.replace('MY_DF col', 'MY_DF').replace('MY_DF name', 'MY_DF').replace('MY_DF id', 'MY_DF')
      comment = comment.replace('Column', f'{arg_replace_list[0]}{df_suffix_str}').replace('pyspark.sql', 'to the default pyspark TimestampType.')
      filter_str = '1_column_1'
      replaced_flag = 'Y'
    # Checks for the occurrence of the word value and replaces with the appropriate standard
    elif 'value' in comment and replaced_flag == 'N':
      if 'value.' in comment:
        comment = re.sub('value.', f"value of {arg_replace_list[0]}{df_suffix_str}.", comment, 1)                 if 'COL_' in arg_replace_list[0] else comment
        comment = re.sub('value.', f"{arg_replace_list[0]}{df_suffix_str}.", comment, 1)                          if f"{arg_replace_list[0]}" not in comment else comment
        comment = comment.replace('value', 'positive value')                                                      if fn_main == 'positive' else comment
        filter_str = '1_value_1'
      elif 'given value' in comment:
        comment = re.sub('given value', f"given value of {arg_replace_list[0]}{df_suffix_str}", comment, 1)       if f"{arg_replace_list[0]}" not in comment else comment
        filter_str = '1_value_2'
      elif 'values' in comment:
        comment = re.sub('in a map', f"in a map given by input {arg_replace_list[0]}{df_suffix_str}", comment, 1) if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('col ', f"{arg_replace_list[0]}{df_suffix_str} ", comment, 1)                            if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('col.', f"{arg_replace_list[0]}{df_suffix_str}.", comment, 1)                            if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('values of a group', f"values in {arg_replace_list[0]}{df_suffix_str}", comment, 1)      if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('values in a group', f"values in {arg_replace_list[0]}{df_suffix_str}", comment, 1)      if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('values', f"values in {arg_replace_list[0]}{df_suffix_str}", comment, 1)                 if f"{arg_replace_list[0]}" not in comment else comment
        filter_str = '1_value_3'
      elif 'value' in comment:
        comment =comment.replace('and returns the value as', 'value and returns')
        comment = re.sub('a predicate', f"a predicate for each element in {arg_replace_list[0]}{df_suffix_str}", comment, 1)    if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('being evaluated', f"being evaluated for {arg_replace_list[0]}{df_suffix_str}", comment, 1)            if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('timestamp string', f"timestamp string in {arg_replace_list[0]}{df_suffix_str}", comment, 1)           if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('string value', f"string value in {arg_replace_list[0]}{df_suffix_str} using", comment, 1)             if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('224 SHA 256 SHA 384 and SHA 512', f"256 for {arg_replace_list[0]}{df_suffix_str} using", comment, 1)  if f"{arg_replace_list[0]}" not in comment else comment

        comment = re.sub('current row', f"current row in {arg_replace_list[0]}{df_suffix_str} using", comment, 1) if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('input using', f"input {arg_replace_list[0]}{df_suffix_str} using", comment, 1)          if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('bucket number', f"bucket number of {arg_replace_list[0]}{df_suffix_str}", comment, 1)   if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('the array', f"{arg_replace_list[0]}{df_suffix_str}", comment, 1)                        if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('value of string', f"value of {arg_replace_list[0]}{df_suffix_str}", comment, 1)         if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('expression in a group', f"{arg_replace_list[0]}{df_suffix_str}", comment, 1)            if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('argument ', f"{arg_replace_list[0]}{df_suffix_str} ", comment, 1)                       if f"{arg_replace_list[0]}" and fn_main == 'rint' not in comment else comment
        comment = re.sub('the text', f"the text in {arg_replace_list[0]}{df_suffix_str}", comment, 1)             if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('same operation', f"same operation on {arg_replace_list[0]}{df_suffix_str}", comment, 1) if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('in a map', f"in a map given by input {arg_replace_list[0]}{df_suffix_str}", comment, 1) if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('col ', f"{arg_replace_list[0]}{df_suffix_str} ", comment, 1)                            if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('col.', f"{arg_replace_list[0]}{df_suffix_str}.", comment, 1)                            if f"{arg_replace_list[0]}" not in comment else comment
        comment = re.sub('value', f"value of {arg_replace_list[0]}{df_suffix_str},", comment, 1)                  if f"{arg_replace_list[0]}" not in comment and 'no match is found' in comment else comment
        comment = re.sub('value', f"value of {arg_replace_list[0]}{df_suffix_str},", comment, 1)                  if f"{arg_replace_list[0]}" not in comment else comment
        filter_str = '1_value_4'
      replaced_flag = 'Y'
    # Checks for the occurrence of the word value and replaces with the appropriate standard
    elif 'argument' in comment and replaced_flag == 'N':
      if fn_main == 'log':
        comment = f"Pyspark code which calculates the logarithm (base 2) of {arg_replace_list[0]}{df_suffix_str}"       if f"{arg_replace_list[0]}" not in comment and '2.0' in code else comment
        comment = f"Pyspark code which calculates the natural logarithm of {arg_replace_list[0]}{df_suffix_str}"        if f"{arg_replace_list[0]}" not in comment and '2.0' not in code else comment
      if 'applyInPandas' in code:
        comment = f"Pyspark code which normalizes the data on {arg_replace_list[0]}{df_suffix_str} using a custom function normalize using pandas"                if f"{arg_replace_list[0]}" not in comment and 'normalize' in code else comment
        comment = f"Pyspark code which calculates the mean of the data in {arg_replace_list[0]}{df_suffix_str} using a custom function normalize using pandas"    if f"{arg_replace_list[0]}" not in comment and 'mean_func' in code else comment
        comment = f"Pyspark code which calculates the sum of the data in {arg_replace_list[0]}{df_suffix_str} using a custom function normalize using pandas"     if f"{arg_replace_list[0]}" not in comment and 'sum_func' in code else comment
        comment = f"Pyspark code which applies the pandas function asof_join for {arg_replace_list[0]}{df_suffix_str}"                                            if f"{arg_replace_list[0]}" not in comment and 'asof_join' in code else comment

      comment = (comment.split('character set')[0] + 'character set UTF-8')                                             if f"{arg_replace_list[0]}" not in comment and 'UTF-8' in code else comment
      comment = re.sub('Splits a string', f"Splits a string given by {arg_replace_list[0]}{df_suffix_str}", comment, 1) if f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub('argument expr', f"{arg_replace_list[0]}{df_suffix_str}", comment, 1)                            if f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub('argument', f"argument given by {arg_replace_list[0]}{df_suffix_str}", comment, 1)               if f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub('argument', f"argument given by {arg_replace_list[0]}{df_suffix_str}", comment, 1)               if f"{arg_replace_list[0]}" not in comment else comment
      filter_str = '1_argument_1'
      replaced_flag = 'Y'
    # Checks for the occurrence of the word array and replaces with the appropriate standard
    elif 'array' in comment:
      comment = re.sub('input array', f'input array {arg_replace_list[0]}{df_suffix_str}', comment, 1)                  if 'input array' in comment else comment
      comment = re.sub('array\.', f'array {arg_replace_list[0]}{df_suffix_str}.', comment, 1)                           if 'array.' in comment and f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub('elements\.', f'elements in {arg_replace_list[0]}{df_suffix_str}.', comment, 1)                  if 'elements.' in comment and f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub('arrays\.', f'arrays in {arg_replace_list[0]}{df_suffix_str}.', comment, 1)                      if 'arrays.' in comment and f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub(' map\.', f' map in {arg_replace_list[0]}{df_suffix_str}.', comment, 1)                          if ' map.' in comment and f"{arg_replace_list[0]}" not in comment else comment
      filter_str = '1_array_1'
      replaced_flag = 'Y'
    # Checks for the occurrence of the word timestamp and replaces with the appropriate standard
    elif 'timestamp' in comment or category == 'Datetime Functions':
      for item in [
          'Parses the timestamp', 'Converts the timestamp', 'given date', 'given timestamp', 'of seconds', 'partition data', 'number of microseconds', 'number of milliseconds', 'time string', 'number of days', 'for date timestamp'
          ]:
        if item in comment and f"{arg_replace_list[0]}" not in comment:
          comment = re.sub(item, f"{item.replace('string str', 'string')} in {arg_replace_list[0]}{df_suffix_str}", comment, 1)

      comment = re.sub('given time\.', f'given time in {arg_replace_list[0]}{df_suffix_str}.', comment, 1)                                                              if 'given time.' in comment and f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub('truncated ', f'truncated from {arg_replace_list[0]}{df_suffix_str} ', comment, 1)                                                               if 'truncated ' in comment and f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub('a timestamp which', f'a timestamp in {arg_replace_list[0]}{df_suffix_str} which', comment, 1)                                                   if 'a timestamp which' in comment and f"{arg_replace_list[0]}" not in comment else comment

      filter_str = '1_time_1'
      replaced_flag = 'Y'
    elif 'str ' in comment or 'str.' in comment:
      for item in ['from str', 'if str', 'in the string str', 'within the string str', 'strings in the str', 'all characters', 'Splits str', 'Decodes a str']:
        if item in comment and f"{arg_replace_list[0]}" not in comment:
          comment = re.sub(item, f"{item.replace('string str', 'string')} in {arg_replace_list[0]}{df_suffix_str}", comment, 1)
      filter_str = '1_str_1'
      replaced_flag = 'Y'
    else:
      comment = comment.replace('which Inverse of hex.', 'which returns the inverse of hex value.')
      in_list   = [
          'Converts an angle', 'a json string', 'a JSON string', 'in a group', 'ank of rows', 'in the sentence', 'string expression', 'for a string', 'Translates a string', 'XPath expression', 'function to each cogroup',
          'for each group'
          ]
      from_list = ['with duplicates', 'elements eliminated', 'lgConfigK arg']
      for_list  = ['starting at 1', 'binary data', 'result of SHA 1', 'with reflection', 'Compute aggregates', 'of hex value']

      for item in in_list + from_list + for_list :
        verb = ''
        if item in comment and f"{arg_replace_list[0]}" not in comment:
          verb = 'in'     if item in in_list else verb
          verb = 'from'   if item in from_list else verb
          verb = 'for'    if item in in_list else verb
          comment = re.sub(item, f"{item} {verb} {arg_replace_list[0]}{df_suffix_str}", comment, 1)
      filter_str = '1_misc_1'

      rplc_dict = {

      }
      if f"{arg_replace_list[0]}" not in comment:
      #, , '', '', 'agg', 'apply', 'applyInPandasWithState', 'applyInPandas', 'avg'
        comment = f"PySpark code which calculates the percentile rank of each row within the specified window partition based on the values in {arg_replace_list[0]}{df_suffix_str}." if fn_main == 'percent_rank' and f"{arg_replace_list[0]}" not in comment else comment
        comment = f"Pyspark code which calculates the {'base-2 logarithm' if 'log(2' in code else  'natural logarithm'} of the values in {arg_replace_list[0]}{df_suffix_str}."       if fn_main == 'log' and f"{arg_replace_list[0]}" not in comment else comment
        comment = f"PySpark code which calculates the variance of the values in {arg_replace_list[0]}{df_suffix_str}."                                                                if fn_main == 'variance' and f"{arg_replace_list[0]}" not in comment else comment
        comment = f"PySpark code which calculates the standard deviation of the values in {arg_replace_list[0]}{df_suffix_str}."                                                      if 'std' in fn_main and f"{arg_replace_list[0]}" not in comment else comment
        comment = comment.replace('input.', f'input {arg_replace_list[0]}{df_suffix_str}.')                                                                                           if fn_main == 'typeof' and f"{arg_replace_list[0]}" not in comment else comment
        comment = f"Pyspark code which groups the data by {arg_replace_list[0]}{df_suffix_str} and counts each group."                                                                if fn_main in ['agg', 'count'] and f"{arg_replace_list[0]}" not in comment else comment
        comment = f"Pyspark code which applies a pandas user defined function 'normalize' to the {arg_replace_list[0]}{df_suffix_str}."                                               if fn_main == 'apply' and f"{arg_replace_list[0]}" not in comment else comment
        comment = comment.replace('each group of the current DataFrame', f'each group of values in {arg_replace_list[0]}{df_suffix_str}')                                             if fn_main == 'applyInPandas' and f"{arg_replace_list[0]}" not in comment else comment

        filter_str = '1_misc_2'

      replaced_flag = 'Y'


# each group of the current DataFrame  -- each group in

  elif replaced_flag == 'N':
    if len(arg_match) == 0:
      filter_str = '0_COLS'

  comment = comment if comment.strip().endswith('.') else f"{comment.strip()}."
  result = [comment, filter_str, comment_orig]
  return result


df_standardize_comments_1 = df_standardize_columns.copy()

# Standardize comments and add a filter column which will be useful for validations
df_standardize_comments_1['code_desc_1']      = df_standardize_columns.apply(lambda x: fn_standardize_comments_1(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[0], axis=1)
df_standardize_comments_1['filter']           = df_standardize_columns.apply(lambda x: fn_standardize_comments_1(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[1], axis=1)
df_standardize_comments_1['code_description'] = df_standardize_columns.apply(lambda x: fn_standardize_comments_1(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[2], axis=1)


# UNCOMMENT TO DEBUG
df_standardize_comments_1[
  #(df_standardize_comments_1['filter'].str.contains('1_INIT'))

  (df_standardize_comments_1['filter'].str.contains('1_misc_2'))

  #(df_standardize_comments_1['filter'].str.contains('=NA='))
#].info()
#].count()
][['function', 'Category', 'code_desc_1', 'code_description', 'code_snippet_1', 'code_snippet', 'filter']].style.set_properties(**{'text-align': 'left'})
#df_standardize_comments_1.style.set_properties(**{'text-align': 'left'})

df_standardize_comments_1.head().style.set_properties(**{'text-align': 'left'})


,code_description,code_snippet,import_line,Category,function,code_snippet_1,code_desc_1,filter
0,Pyspark code to Write a DataFrame into a CSV file.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",Pyspark code to Write a DataFrame into a CSV file.The data is written to file or directory MY_DIR.,1_MY_DIR
1,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.The data is written to file or directory MY_DIR.,1_MY_DIR
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'Hyukjin Kwon'.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")",Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'LIT_STR_0'.The data is read from file or directory MY_DIR.,1_MY_DIR
3,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")",Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.The data is read from file or directory MY_DIR.,1_MY_DIR
4,Pyspark code to Write a DataFrame into a JSON file.,"df.write.mode(""overwrite"").format(""json"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.format,"MY_DF.write.mode(""overwrite"").format(""json"").save(MY_DIR)",Pyspark code to Write a DataFrame into a JSON file.The data is written to file or directory MY_DIR.,1_MY_DIR


In [11]:
# HANDLE COMMENTS PASS 1: validate how many rows per filter category were standardized

fn_display_header("HANDLE COMMENTS PASS 1: validate how many rows per filter category were standardized")

df_standardize_comments_1.groupby('filter').size().reset_index(name='count')

--------------------------------------------------------------------------------
           HANDLE COMMENTS PASS 1: validate how many rows per filter category were standardized
--------------------------------------------------------------------------------


,filter,count
0,0_COLS,37
1,1_MY_DIR,90
2,1_argument_1,13
3,1_array_1,17
4,1_column_1,78
5,1_fn_explode_1,5
6,1_misc_1,30
7,1_misc_2,14
8,1_str_1,29
9,1_time_1,46


<br>
<br>
<br>

----
# <b>CODE/COMMENT STANDARDIZATION: PASS 2</b>
----
<br>

In [12]:
# HANDLE COMMENTS PASS 2: Manipulate comments to incorporate standards introduced in code
#                         Handles double occurrences of COL or LITERAL i.e. functions passing 2 arguments

# Function to handle functions passing 2 arguments
def fn_standardize_comments_2(comment, code, function, category):
  fn_main = function.split('.')[-1].strip()
  filter_str = '=NA='
  replaced_flag = 'N'

  # Handle some special cases for comments
  if not comment.strip().endswith('.'):
    comment = comment + '.'

  comment = comment.replace('Aggregate function', '').replace('Collection function', '').replace('Window function', '')
  comment = comment.replace('thesame', 'the same').replace('valuein', 'value in').replace('columnPyspark', 'column Pyspark').replace('andcol2', 'and col2').replace('forcolumn', 'for column').replace('aswell', 'as well')
  comment = comment.replace('delimitedlist', 'delimited list').replace('pairsin', 'pairs in').replace('srcCol', 'source Col').replace('isreturned', 'is returned').replace('leftmost', 'left most').replace('rightmost', 'right most')
  comment = comment.replace('placeswith',  'places with').replace('sensitiveinformation', 'sensitive information')

  comment_orig = comment

  if fn_main in delim_dict:
    comment = comment.replace('delimiter', 'delimiter ' + delim_dict[fn_main])

  # search code for occurrences of standardized column names and literals
  arg_match = re.findall(r"(LIT_(INT|DEC|STR)_\d+|COL_[A-Z]|YEAR|MONTH|DAY|HOUR|MIN|SEC)", code)
  arg_match = list(dict.fromkeys([match[0] for match in arg_match]))

  arg_replace_list = []

  for item in arg_match:
    if 'COL_' in item or item in ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MIN', 'SEC']:
      arg_replace_list.append(f"column {item}")
    else:
      arg_replace_list.append(f"{lit_search_id_dict[item]} value {item}")



  exception_functions = ['']
  if len(arg_match) == 2 and replaced_flag == 'N':
    df_suffix_str = " in dataframe MY_DF"   #if 'COL_' in code else ''

    if 'COL_' in arg_replace_list[0]:
      arg_replace_list[0] = arg_replace_list[0] + df_suffix_str
    if 'COL_' in arg_replace_list[1]:
      arg_replace_list[0] = arg_replace_list[0].replace(df_suffix_str, '')
      arg_replace_list[1] = arg_replace_list[1] + df_suffix_str

    REPL_0 = arg_replace_list[0]
    REPL_1 = arg_replace_list[1]

    #if 'float' not in fn_main:
    #  comment = comment.replace('float', ' ')

    if 'dividend' in comment:
      comment = comment.replace('positive value of dividend mod divisor', 'positive mod value of dividend divisor')
      comment = comment.replace("dividend divisor", f"the result of division where the numerator is {REPL_0} and denominator is {REPL_1}")
      filter_str = '2_ARG_1'
      replaced_flag = 'Y'
    elif 'Returns left right' in comment:
      comment = f"Pyspark code which multiplies the value of {REPL_0} with {REPL_1}"                        if f"{arg_replace_list[0]}" not in comment and fn_main == 'try_multiply' else comment
      if fn_main == 'try_subtract':
        comment = f"Pyspark code which subtract the value of {REPL_1} from {REPL_0}"                                                                                              if f"{arg_replace_list[0]}" not in comment and 'make_interval' not in code else comment
        comment = f"Pyspark code which subtract the value of interval duration in {REPL_1} from {REPL_0}. Uses the make_interval function for deriving the interval in {REPL_1}"  if f"{arg_replace_list[0]}" not in comment and 'make_interval' in code else comment
      filter_str = '2_ARG_2'
      replaced_flag = 'Y'
    elif fn_main == 'try_add':
      comment = f"Pyspark code which adds the value of {REPL_0} to {REPL_1}"                                                                                                  if f"{arg_replace_list[0]}" not in comment and 'make_interval' not in code else comment
      comment = f"Pyspark code which adds the value of {REPL_0} to the interval duration in {REPL_1}. Uses the make_interval function for deriving the interval in {REPL_1}"  if f"{arg_replace_list[0]}" not in comment and 'make_interval' in code else comment
      filter_str = '2_ARG_3'
      replaced_flag = 'Y'
    elif 'value in' in comment:
      comment = re.sub('given value in the given array', f"element {REPL_1} in the array {REPL_0}", comment, 1)         if f"{arg_replace_list[0]}" not in comment else comment
      comment = re.sub('value in a group', f"value of {REPL_1} grouped by {REPL_0}", comment, 1)                        if f"{arg_replace_list[0]}" not in comment else comment
      filter_str = '2_ARG_4'
      replaced_flag = 'Y'
    elif 'date_add' in code or 'add_months' in code or 'date_sub' in code or 'dateadd' in code:
      comment = f"Pyspark code which adds the the number of months given by {REPL_1} to the date {REPL_0}"              if f"{arg_replace_list[0]}" not in comment and 'add_months' in code else comment
      comment = f"Pyspark code which adds the the number of days given by {REPL_1} to the date {REPL_0}"                if f"{arg_replace_list[0]}" not in comment and 'date_add' in code else comment
      comment = f"Pyspark code which adds the the number of days given by {REPL_1} to the date {REPL_0}"                if f"{arg_replace_list[0]}" not in comment and 'dateadd' in code else comment
      comment = f"Pyspark code which subtracts the the number of days given by {REPL_1} to the date {REPL_0}"           if f"{arg_replace_list[0]}" not in comment and 'date_sub' in code else comment
      filter_str = '2_ARG_5'
      replaced_flag = 'Y'
    elif 'element of array' in comment:
      comment = comment.replace('array index', '')
      if 'get' in code:
        comment = re.sub('element of array', f"element of array in {REPL_0}", comment, 1)
        comment = re.sub('index', f"index given by {REPL_1}.##", comment, 1).split('##')[0]
      comment = re.sub('index', f"index given by {REPL_1} in array or map {REPL_0}.##", comment, 1).split('##')[0]      if f"{arg_replace_list[0]}" not in comment and 'element_at' in code else comment
      comment = f"Pyspark code which adds the the number of months given by {REPL_1} to the date {REPL_0}"              if f"{arg_replace_list[0]}" not in comment and 'add_months' in code else comment
      filter_str = '2_ARG_6'
      replaced_flag = 'Y'
    elif 'input column' in comment:
      comment = re.sub('input columns', f"input columns {REPL_1} and {REPL_0}", comment, 1)                             if f"{arg_replace_list[0]}" not in comment and 'input columns' in comment else comment
      if fn_main in ['assert_true']:
        comment = f"Pyspark code which Returns null if the condition {REPL_0} < {REPL_1} is true."                      if '<' in code else comment
        comment = f"Pyspark code which Returns null if the condition {REPL_0} < {REPL_1} is true."                      if '>' in code else comment
        for default_val in [', MY_DF.COL_A', ", 'error", ", 'My error msg"]:
          if default_val in code:
            default_val = default_val.replace(', ', '').replace("'", '')
            default_val = f"'{default_val}'" if 'COL_' not in default_val else f"in column {default_val}"
            comment =  comment + f" The function throws an exception with error message {default_val}."
      filter_str = '2_ARG_7'
      replaced_flag = 'Y'
    elif 'columns' in comment:
      comment = comment.replace('one column Pyspark code which Two or more columns', 'numeric columns')
      comment = re.sub('string columns', f"string columns {REPL_0} and {REPL_1}", comment, 1).replace('separator', "separator '-'")   if f"{arg_replace_list[0]}" not in comment and 'string columns' in comment else comment
      comment = re.sub('given columns', f"given columns {REPL_0} and {REPL_1}", comment, 1)                                           if f"{arg_replace_list[0]}" not in comment and 'given columns' in comment else comment
      if ("'COL_B')" in code or '"COL_B")' in code) and 'numeric columns for each group' in comment:
        comment = re.sub('each numeric columns', f"numeric {REPL_1}", comment, 1)
        comment = re.sub('for each group', f"grouped by {REPL_0}", comment, 1)
      else:
        comment = re.sub('each numeric columns', f"numeric columns {REPL_0} and {REPL_1}", comment, 1)
        comment = re.sub('for each group', f"grouped by the entire dataframe MY_DF", comment, 1)
      comment = re.sub('numeric columns', f"numeric columns {REPL_0} and {REPL_1}", comment, 1)     if f"{arg_replace_list[0]}" not in comment and fn_main == 'xxhash64' else comment
      filter_str = '2_ARG_8'
      replaced_flag = 'Y'
    elif 'col1' in comment:
      comment = comment.replace('col1', f"{REPL_0}").replace('col2', f"{REPL_1}")
      filter_str = '2_ARG_9'
      replaced_flag = 'Y'
    elif 'array' in comment:
      comment = comment.replace('array column', f"array column from {REPL_0} and {REPL_1}")
      if fn_main in ['array_contains']:
        array_verb = f"{fn_main.split('_')[1]}s".replace('ss', 's')
        comment = f"Pyspark code which checks if the array {REPL_0} {array_verb} the value of {REPL_1}."
      elif fn_main in ['arrays_overlap']:
        array_verb = f"{fn_main.split('_')[1]}s".replace('ss', 's').replace('overlaps', 'overlaps or exists in')
        comment = f"Pyspark code which checks if the value of {REPL_1} {array_verb} the array {REPL_0}."
      elif fn_main in ['array_insert', 'array_remove', 'array_prepend']:
        array_verb = f"{fn_main.split('_')[1]}s".replace('ss', 's')
        comment = f"Pyspark code which {array_verb} the value of {REPL_1} the array {REPL_0}."

      comment = comment.replace('input array', f"input array {REPL_0} ")                                                              if"{arg_replace_list[0]}" not in comment and fn_main == 'array_sort' else comment
      comment = comment.replace('a column repeated count times', f"a column {REPL_0} repeated the number of times in {REPL_1}")       if f"{arg_replace_list[0]}" not in comment and fn_main == 'array_repeat' else comment
      comment = comment.replace('two arrays', f"array {REPL_0} and array {REPL_1}")                                                   if f"{arg_replace_list[0]}" not in comment and fn_main == 'map_from_arrays' else comment
      comment = comment.replace('two given arrays', f"array {REPL_0} and array {REPL_1}").replace('function', 'lambda function')      if f"{arg_replace_list[0]}" not in comment and 'element wise' in comment else comment
      comment = f"Pyspark code which aggregates the {REPL_0} using initial accumulator {REPL_1} via a lambda function."               if f"{arg_replace_list[0]}" not in comment and fn_main == 'aggregate' else comment
      comment = f"Pyspark code which computes a histogram for the numeric values in {REPL_0} with number of bins given by {REPL_1}."  if f"{arg_replace_list[0]}" not in comment and fn_main == 'histogram_numeric' else comment
      comment = f"Pyspark code which Returns the substring of str in {REPL_0} that starts at position given by {REPL_1}."             if f"{arg_replace_list[0]}" not in comment and fn_main == 'substr' else comment
      comment = f"Pyspark code which uses a reduce function to sum up the values of {REPL_0} using initial accumulator {REPL_1} via a lambda functon."  if f"{arg_replace_list[0]}" not in comment and fn_main == 'reduce' else comment

      filter_str = '2_ARG_10'
      replaced_flag = 'Y'
    elif 'group' in comment:
      if '_value(' in code:
        comment = comment.replace('some value', 'any value').replace('of col for a group of rows', f'of {REPL_0} and {REPL_1}')
        comment = comment.replace('the values', f'the values in {REPL_1}').replace('in a group', f'grouped by {REPL_0}')                if f"{arg_replace_list[0]}" not in comment and 'median' in comment else comment
        comment = comment.replace('c1 n 1 grouping c2 n 2 grouping cn', f'c1 n 1 grouping c2 n 2 grouping cn')                          if f"{arg_replace_list[0]}" not in comment and 'median' in comment else comment
        filter_str = '2_ARG_11'
      elif 'groupBy' in code or 'groupby' in code:
        rplc_dict = {'min': 'minimum', 'min_udf': 'minimum', 'avg': 'average', 'max': 'maximum', 'sum': 'sum', 'median': 'median'}
        comment = f"Pyspark code which groups by on {REPL_0 if 'groupBy()' not in code else 'entire dataset MY_DF'} and calculates the {[ rplc_dict[x] for x in rplc_dict.keys() if x + '(' in code][0]} of {REPL_1}." #+ comment
        filter_str = '2_ARG_11'
      elif 'regr_' in fn_main:
        rplc_dict = {
            'regr_avgx': 'average of the independent variable x for non-null pairs in a group',
            'regr_avgy': 'average of the dependent variable y for non-null pairs in a group',
            'regr_count': 'count non-null number pairs used to fit the regression line',
            'regr_intercept': 'y-intercept of the regression line',
            'regr_r2': 'coefficient of determination (R-squared) for the regression',
            'regr_slope': 'slope of the regression line',
            }
        if fn_main not in ['regr_sxx', 'regr_sxy', 'regr_syy']:
          comment = f"Pyspark code which returns the {[ rplc_dict[x] for x in rplc_dict.keys() if x + '(' in code][0]} where x in the independent variable given by {REPL_0} and y is the dependent variable given by {REPL_1}." #+ '-----' + comment
        filter_str = '2_ARG_11' if fn_main not in ['regr_sxx', 'regr_sxy', 'regr_syy'] else '2_ARG_11_IGNORE'
      else:
        filter_str = '2_ARG_11_IGNORE'

      replaced_flag = 'Y'

    elif 'Math' in category:
      comment = comment.replace('given column', f'given columns {REPL_0} and {REPL_1}')
      comment = comment.replace('sqrt a 2 b 2', f'square root of the squared values of {REPL_0} and {REPL_1}')
      comment = comment.replace('first argument', f'first argument {REPL_0}').replace('second argument', f'second argument {REPL_1}')
      comment = comment.replace('given value', f'given value {REPL_0}').replace('numBits', f'number of bits given by {REPL_1} towards')     if 'numBits' in comment else comment
      comment = comment.replace('given value', f'{REPL_0} and {REPL_1}')                                                                    if 'numBits' not in comment else comment
      filter_str = '2_ARG_12_A'
      replaced_flag = 'Y'

    elif 'Datetime' in category:
      comment = comment.replace('date1', f'{REPL_0}').replace('date2', f'{REPL_1}')
      comment = comment.split('from')[0] + f"from columns {REPL_0} {'and ' + REPL_1 if 'WEEK' not in code else ',WEEK and ' + REPL_1} in dataframe MY_DF".replace('column ', '')      if 'interval' in fn_main else comment
      comment = f"Pyspark code which Returns the difference of the date columns {REPL_0} and {REPL_1}"                                                                                if 'diff' in fn_main else comment

      filter_str = '2_ARG_12_B'         if 'timestamp' not in fn_main else '2_ARG_12_B_IGNORE'
      replaced_flag = 'Y'
    elif 'Collection' in category:
      comment = comment.replace('column', f'column with {REPL_0} and {REPL_1}')
      comment = comment.replace('map contains the key', f'map column {REPL_0} contains the key value given by {REPL_1}')
      comment = comment.replace('given maps', f'given map columns {REPL_0} and {REPL_1}').replace('column ', '')
      comment = comment.replace('from start to stop incrementing by step', f'starting from value in {REPL_0} to the value in {REPL_1} incrementing by step 1 if not provided').split('.')[0]
      filter_str = '2_ARG_12_C'
      replaced_flag = 'Y'
    elif 'Aggregate' in category:
      if 'groupby' in code:
        rplc_dict = {'first': 'first value', 'last': 'last value'}
        comment = f"Pyspark code which groups by {REPL_0} and calculates the {[ rplc_dict[x] for x in rplc_dict.keys() if x + '(' in code][0]} of {REPL_1}." #+ comment
      else:
        comment = comment.replace('percentile s', 'percentiles').replace('percentage s', 'percentages').replace('numeric column expr', f'numeric {REPL_0}')
      filter_str = '2_ARG_12_D'         if fn_main != 'hll_sketch_agg' else '2_ARG_12_D_IGNORE'
      replaced_flag = 'Y'
    elif 'String' in category:
      if 'boolean' in comment:
        comment = comment.split('.')[0] + f" if {REPL_0} {fn_main} the value of {REPL_1}.".replace('with', ' with')
        #filter_str = '2_ARG_12_E'
        replaced_flag = 'Y'
      elif 'substr' in comment:
        comment = comment.replace('substr column', 'substr ').replace('occurrence of substr', f'occurrence of {REPL_1}').replace(' in a string', f' in {REPL_0}').replace(' in str ', f' in {REPL_0}').replace('given string', f'given string {REPL_0}')
        comment = comment.replace('after position start', '').replace(' column after position pos', '')
        comment = comment.replace('from string str', f"from {REPL_0}").replace('delimiter delim', "delimiter '.'").replace('before count', 'before number of')
        filter_str = '2_ARG_12_E'
        replaced_flag = 'Y'
      else:
        comment = comment.replace('two given strings', f'two given strings in {REPL_0} and {REPL_1}')
        comment = comment.replace('from str', f'from {REPL_0} using {REPL_1} as the trim character.')                                                               if fn_main == 'btrim' else comment
        comment = comment.replace('string st', f'{REPL_0}').replace('list strArray', f'string Array given by {REPL_1}' )                                            if fn_main == 'find_in_set' else comment
        comment = f"Pyspark code which pads the given string '#' to the {'left' if fn_main == 'lpad' else 'right'} of {REPL_0} up to a total width of {REPL_1}"     if 'pad' in fn_main else comment
        comment = f"Pyspark code which formats the number in {REPL_0} to a specific pattern rounded to the number of decimal places given by {REPL_1}."             if fn_main == 'format_number' else comment
        comment = comment.replace('the arguments', f'the {REPL_0} and {REPL_1} in the dataframe df')                                                                if fn_main == 'format_string' else comment
        comment = f'Pyspark code which returns the substring from the {fn_main} of {REPL_0} based on the length of the value in {REPL_1}.'                          if fn_main in ['left', 'right'] else comment
        comment = comment.replace('string column n times', f'the string in {REPL_0} the number of times given by {REPL_1}')                                         if fn_main == 'repeat' else comment
        comment = comment.replace('given string value', f'given string in {REPL_0} with {REPL_1} to replace upper-case characters with')                            if fn_main == 'mask' else comment
        comment = comment.replace('URL', f'URL where column {REPL_0} is the URL string and column {REPL_1} is extract part (like HOST, PATH, QUERY etc)')           if fn_main == 'parse_url' else comment
        comment = comment.replace('search with replace', f'{REPL_0} with {REPL_1}')                                                                                 if fn_main == 'replace' else comment
        if 'like' in fn_main:
          comment = f"Pyspark code which checks if the value of {REPL_0} is (case {'sensitive' if fn_main == 'like' else 'insensitive'}) similar to any of the comma-separated string values in {REPL_1}. It returns a boolean column if a match is found. Default escape character is blank ''."
        filter_str = '2_ARG_12_F'
        replaced_flag = 'Y'

    elif replaced_flag == 'N':
      comment = comment.replace('of input', f'of the input {REPL_0}').replace('with padding', f'with key {REPL_1}')                     if fn_main in ['aes_decrypt', 'aes_encrypt'] else comment
      comment = comment.replace('input bitmap', f'input bitmap of the input {REPL_0}')                                                  if fn_main == 'bitmap_count' else comment
      comment = comment.replace('specified position', f'specified position given by {REPL_1} from  {REPL_0}')                           if fn_main in ['bit_get', 'getbit'] else comment
      comment = comment.replace('which Returns', f'which evalutes the columns {REPL_0} and {REPL_1} and returns')                       if fn_main == 'equal_null' else comment
      comment = f"Pyspark code which returns the value of {REPL_0} for the row number given by {REPL_1} in the window frame."           if fn_main == 'nth_value' else comment
      comment = f"Pyspark code which returns the value of {REPL_0} for the previous row number given by {REPL_1} in the window frame."  if fn_main == 'lag' else comment
      filter_str = '2_ARG_13' if fn_main not in ['reflect'] else '2_ARG_13_IGNORE'
      replaced_flag = 'Y'


  comment = comment if comment.strip().endswith('.') else f"{comment.strip()}."
  result = [comment, filter_str, comment_orig]
  return result


df_standardize_comments_2 = df_standardize_comments_1.copy()

# Standardize Column names calling the function with level 3 first then 2 and then 1
df_standardize_comments_2['code_desc_1']      = df_standardize_comments_2.apply(
                                                    lambda x: fn_standardize_comments_2(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[0]   if x['filter'] == '=NA=' else x['code_desc_1']
                                                              , axis=1)
df_standardize_comments_2['filter']           = df_standardize_comments_2.apply(
                                                    lambda x: fn_standardize_comments_2(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[1]   if x['filter'] == '=NA=' else x['filter'], axis=1)
df_standardize_comments_2['code_description'] = df_standardize_comments_2.apply(
                                                    lambda x: fn_standardize_comments_2(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[2]   , axis=1)

# UNCOMMENT TO DEBUG
df_standardize_comments_2[
  #(df_standardize_comments_2['filter'].str.contains('2_ARG_1'))
  #(df_standardize_comments_2['filter'].str.contains('2_ARG_10'))
  #(df_standardize_comments_2['filter'].str.contains('2_ARG_12_F'))

  (df_standardize_comments_2['filter'].str.contains('2_ARG_13'))

  #(df_standardize_comments_2['filter'].str.contains('=NA='))
#].info()
#].count()
#][['filter', 'code_desc_1', 'code_snippet_1', 'code_description', 'code_snippet', 'function', 'Category' ]].style.set_properties(**{'text-align': 'left'})
][['function', 'Category', 'code_desc_1', 'code_snippet_1', 'code_snippet', 'filter' ]].style.set_properties(**{'text-align': 'left'})

#df_standardize_comments_2.style.set_properties(**{'text-align': 'left'})

df_standardize_comments_2.head().style.set_properties(**{'text-align': 'left'})


,code_description,code_snippet,import_line,Category,function,code_snippet_1,code_desc_1,filter
0,Pyspark code to Write a DataFrame into a CSV file.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",Pyspark code to Write a DataFrame into a CSV file.The data is written to file or directory MY_DIR.,1_MY_DIR
1,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.The data is written to file or directory MY_DIR.,1_MY_DIR
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'Hyukjin Kwon'.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")",Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'LIT_STR_0'.The data is read from file or directory MY_DIR.,1_MY_DIR
3,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")",Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.The data is read from file or directory MY_DIR.,1_MY_DIR
4,Pyspark code to Write a DataFrame into a JSON file.,"df.write.mode(""overwrite"").format(""json"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.format,"MY_DF.write.mode(""overwrite"").format(""json"").save(MY_DIR)",Pyspark code to Write a DataFrame into a JSON file.The data is written to file or directory MY_DIR.,1_MY_DIR


In [13]:
# HANDLE COMMENTS PASS 2: validate how many rows per filter category were standardized

fn_display_header("HANDLE COMMENTS PASS 2: validate how many rows per filter category were standardized")

df_standardize_comments_2.groupby('filter').size().reset_index(name='count')

--------------------------------------------------------------------------------
           HANDLE COMMENTS PASS 2: validate how many rows per filter category were standardized
--------------------------------------------------------------------------------


,filter,count
0,0_COLS,37
1,1_MY_DIR,90
2,1_argument_1,13
3,1_array_1,17
4,1_column_1,78
5,1_fn_explode_1,5
6,1_misc_1,30
7,1_misc_2,14
8,1_str_1,29
9,1_time_1,46


<br>
<br>
<br>

----
# <b>CODE/COMMENT STANDARDIZATION: PASS 3</b>
----
<br>

In [14]:
# HANDLE COMMENTS PASS 3: Manipulate comments to incorporate standards introduced in code
#                         Handles tripple occurrences of COL or LITERAL i.e. functions passing 3 arguments

# Function to handle functions passing 2 arguments
def fn_standardize_comments_2(comment, code, function, category):
  fn_main = function.split('.')[-1].strip()
  filter_str = '=NA='
  replaced_flag = 'N'

  # Handle some special cases for comments
  if not comment.strip().endswith('.'):
    comment = comment + '.'

  comment = comment.replace('Aggregate function', '').replace('Collection function', '').replace('Window function', '')
  comment = comment.replace('resultby',  'result by').replace('anddefault', 'and default').replace('sensitiveinformation', 'sensitive information')

  comment_orig = comment

  if fn_main in delim_dict:
    comment = comment.replace('delimiter', 'delimiter ' + delim_dict[fn_main])

  # search code for occurrences of standardized column names and literals
  arg_match = re.findall(r"(LIT_(INT|DEC|STR)_\d+|COL_[A-Z]|YEAR|MONTH|DAY|HOUR|MIN|SEC)", code)
  arg_match = list(dict.fromkeys([match[0] for match in arg_match]))

  arg_replace_list = []

  for item in arg_match:
    if 'COL_' in item or item in ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MIN', 'SEC']:
      arg_replace_list.append(f"column {item}")
    else:
      arg_replace_list.append(f"{lit_search_id_dict[item]} value {item}")



  exception_functions = ['']
  if len(arg_match) == 3 and replaced_flag == 'N':
    df_suffix_str = " in dataframe MY_DF"   #if 'COL_' in code else ''

    if 'COL_' in arg_replace_list[0]:
      arg_replace_list[0] = arg_replace_list[0] + df_suffix_str
    if 'COL_' in arg_replace_list[1]:
      arg_replace_list[0] = arg_replace_list[0].replace(df_suffix_str, '')
      arg_replace_list[1] = arg_replace_list[1] + df_suffix_str
    if 'COL_' in arg_replace_list[2]:
      arg_replace_list[1] = arg_replace_list[1].replace(df_suffix_str, '')
      arg_replace_list[2] = arg_replace_list[2] + df_suffix_str

    REPL_0 = arg_replace_list[0]
    REPL_1 = arg_replace_list[1]
    REPL_2 = arg_replace_list[2]

    if replaced_flag == 'N' :
      comment = comment.replace("dividend divisor", f"the result of division where the numerator is {REPL_0} and denominator is {REPL_1} {df_suffix_str}")                            if 'dividend' in comment else comment
      comment = comment.replace('column', f'{REPL_0}').replace('one base to another.', f'base in {REPL_1} to base in {REPL_2}.')                                                      if fn_main == 'conv' else comment
      comment = comment.replace('two given strings', f'two given strings in {REPL_0} and {REPL_1} where {REPL_2} gives the maximum edit distance')                                    if fn_main == 'levenshtein' else comment
      comment = f'Pyspark code which returns an array containing all the elements in {REPL_0} with starting index as {REPL_1} and length {REPL_2}'                                    if fn_main == 'slice' else comment
      comment = comment.replace('input columns', f"input columns {REPL_0}, {REPL_1} and {REPL_2}")                                                                                    if fn_main == 'concat' else comment
      comment = comment.replace('adds an item', f'inserts value in {REPL_2}').replace('a given array', f"array {REPL_0}").replace('array index', f'array index given by {REPL_1}')    if fn_main == 'array_insert' else comment
      comment = f'Pyspark code which creates an interval column by combining the values from the DAY, HOUR and MIN columns {df_suffix_str}.'                                          if fn_main == 'make_dt_interval' else comment
      comment = f'Pyspark code which creates an interval column by combining the values from the YEAR, MONTH, WEEK and DAY columns {df_suffix_str}.'                                  if fn_main == 'make_interval' else comment
      comment = f'Pyspark code which creates a new column with dates built from the values in the YEAR, MONTH and DAY columns {df_suffix_str}.'                                       if fn_main == 'make_date' else comment
      comment = comment.replace('from start to stop incrementing by step', f'starting from {REPL_0} to {REPL_1} incrementing by step given by {REPL_2}')                              if fn_main == 'sequence' else comment
      comment = f'Pyspark code which creates a new array column by merging the values from the specified columns {REPL_0}, {REPL_1} and {REPL_2}'                                     if fn_main == 'arrays_zip' else comment
      comment = comment.replace('a column with the given esp confidence and seed', f'{REPL_0} with the given esp confidence given by {REPL_1} and seed given by {REPL_2}')            if fn_main == 'count_min_sketch' else comment
      comment = comment.replace('col2', f'{REPL_1}').replace('col1', f'{REPL_0}').replace('col3', f'{REPL_2}')                                                                        if fn_main == 'nvl2' else comment
      comment = comment.replace('a column', f'{REPL_1}').replace('aggregation', f' aggregation on {REPL_2} grouped by values in {REPL_1}')                                            if fn_main == 'pivot' else comment
      comment = f"Pyspark code which Compute the sum of {REPL_2} for each value of {REPL_0} pivoted on {REPL_1}."                                                                     if fn_main == 'pivot' and 'which Compute' in comment else comment
      comment = comment.replace('in the array', f'in the array {REPL_0}')                                                                                                             if fn_main == 'reduce' else comment
      comment = comment.replace('the value', f'the value in {REPL_0}').replace('offset rows', f'offset rows given by {REPL_1}').replace('default', f'default to {REPL_2}')            if fn_main == 'lead' else comment
      comment = f"Returns the n-th input where n is given by {REPL_0}. Returns {REPL_1} when n is 1 and {REPL_2} when n is 2"                                                         if fn_main == 'elt' else comment
      comment = comment.replace('URL.', f'URL where {REPL_0} is the URL string and {REPL_1} is extract part (like HOST, PATH, QUERY etc) with the key as {REPL_2}')                   if fn_main == 'parse_url' else comment
      comment = comment.replace('arguments', f'arguments {REPL_1} and {REPL_2}').replace('printf style', f'printf style given by the format in {REPL_0}')                             if fn_main == 'printf' else comment
      comment = comment.replace('specified string', f'specified string {REPL_0}').replace('regexp with replacement', f'regular expression in {REPL_1} with replacement in {REPL_2}')  if fn_main == 'regexp_replace' else comment
      comment = comment.replace('occurrences of', f'occurrences of {REPL_0}').replace('search with replace', f'of search string in {REPL_1} with replacement in {REPL_2}')            if fn_main == 'replace' else comment
      comment = comment.replace('Splits str', f'splits string in {REPL_0}').replace('delimiter', f'delimiter {REPL_1}').replace(' part', f' part given by {REPL_2}')                  if fn_main == 'split_part' else comment
      comment = f"Pyspark code which Returns the substring of string in {REPL_0} that starts at position given by {REPL_1} and is of length given by {REPL_2}"                        if fn_main in ['substr', 'substring'] else comment
      comment = f"Pyspark code which overlay the specified portion of value in {REPL_0} with replace string in {REPL_1} starting from byte position {REPL_2}."                        if fn_main == 'overlay' else comment
      comment = comment.replace('a string', f'a string in {REPL_0}').replace('language', f'language given by {REPL_1}').replace('country', f'country given by {REPL_2}')              if fn_main == 'sentences' else comment
      comment = comment.replace('A function', '').replace('the srcCol', f'in {REPL_0}').replace('in matching', f'in {REPL_1} with {REPL_2}').split('.')[0]                            if fn_main == 'translate' else comment
      comment = comment.replace('of input', f'of the input {REPL_0}').replace('using AES in mode', f'using mode in {REPL_1}').replace('with padding', f'with key {REPL_2}')           if fn_main in ['aes_decrypt', 'try_aes_decrypt'] else comment

      comment = comment.replace('given array or map', f'array or map given by {REPL_2} {df_suffix_str}') + f". It includes {REPL_0} and {REPL_1} in the explode process."                   if 'explode' in fn_main else comment
      comment = comment.replace('given string value', f'given string in {REPL_0} with {REPL_1} to replace upper-case characters with and {REPL_2} to replace lower-case characters with')   if fn_main == 'mask' else comment

      if fn_main in ['max_by', 'min_by']:
        rplc_dict = {'max_by': 'maximum', 'min_by': 'minimum'}
        comment = f"Pyspark code which groups by on {REPL_0} and calculates the {[ rplc_dict[x] for x in rplc_dict.keys() if x + '(' in code][0]} of {REPL_1} and {REPL_2}." #+ comment

      filter_str = '3_ARG_1' if fn_main not in ['aggregate', 'grouping_id'] else '3_ARG_1_IGNORE'
      replaced_flag = 'Y'

  comment = comment if comment.strip().endswith('.') else f"{comment.strip()}."
  result = [comment, filter_str, comment_orig]
  return result


df_standardize_comments_3 = df_standardize_comments_2.copy()

# Standardize Column names calling the function with level 3 first then 2 and then 1
df_standardize_comments_3['code_desc_1']      = df_standardize_comments_3.apply(
                                                    lambda x: fn_standardize_comments_2(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[0]  if x['filter'] == '=NA=' else x['code_desc_1'], axis=1)
df_standardize_comments_3['filter']           = df_standardize_comments_3.apply(
                                                    lambda x: fn_standardize_comments_2(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[1]  if x['filter'] == '=NA=' else x['filter'], axis=1)
df_standardize_comments_3['code_description'] = df_standardize_comments_3.apply(
                                                    lambda x: fn_standardize_comments_2(x['code_description'], x['code_snippet_1'], x['function'], x['Category'])[2]  , axis=1)

# UNCOMMENT TO DEBUG
df_standardize_comments_3[
  (df_standardize_comments_3['filter'].str.contains('3_ARG'))
  #(df_standardize_comments_3['filter'].str.contains('=NA='))
#].info()
#].count()
#][['filter', 'code_desc_1', 'code_snippet_1', 'code_description', 'code_snippet', 'function', 'Category' ]].style.set_properties(**{'text-align': 'left'})
][['function', 'Category', 'code_desc_1', 'code_snippet_1', 'code_snippet', 'filter' ]].style.set_properties(**{'text-align': 'left'})

#df_standardize_comments_3.style.set_properties(**{'text-align': 'left'})

df_standardize_comments_3.head().style.set_properties(**{'text-align': 'left'})


,code_description,code_snippet,import_line,Category,function,code_snippet_1,code_desc_1,filter
0,Pyspark code to Write a DataFrame into a CSV file.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",Pyspark code to Write a DataFrame into a CSV file.The data is written to file or directory MY_DIR.,1_MY_DIR
1,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.The data is written to file or directory MY_DIR.,1_MY_DIR
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'Hyukjin Kwon'.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")",Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'LIT_STR_0'.The data is read from file or directory MY_DIR.,1_MY_DIR
3,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")",Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.The data is read from file or directory MY_DIR.,1_MY_DIR
4,Pyspark code to Write a DataFrame into a JSON file.,"df.write.mode(""overwrite"").format(""json"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.format,"MY_DF.write.mode(""overwrite"").format(""json"").save(MY_DIR)",Pyspark code to Write a DataFrame into a JSON file.The data is written to file or directory MY_DIR.,1_MY_DIR


In [15]:
# HANDLE COMMENTS PASS 3: validate how many rows per filter category were standardized

fn_display_header("HANDLE COMMENTS PASS 3: validate how many rows per filter category were standardized")

df_standardize_comments_3.groupby('filter').size().reset_index(name='count')

--------------------------------------------------------------------------------
           HANDLE COMMENTS PASS 3: validate how many rows per filter category were standardized
--------------------------------------------------------------------------------


,filter,count
0,0_COLS,37
1,1_MY_DIR,90
2,1_argument_1,13
3,1_array_1,17
4,1_column_1,78
5,1_fn_explode_1,5
6,1_misc_1,30
7,1_misc_2,14
8,1_str_1,29
9,1_time_1,46


<br>
<br>
<br>

----
# <b>FINALIZE: APPEND PRE AND POST STANDARDIZATION DATASETS</b>
----
<br>

In [16]:
# RECREATE THE PRE STANDARDIZATION DATASET WITH DESIRED COLUMNS

df_pre_standardization = df_standardize_comments_3[['code_description', 'code_snippet', 'import_line', 'Category', 'function']].copy()

# Add a column to identify the original records
df_pre_standardization['origin_str'] = 'original'

fn_display_header('PRE-STANDADIZATION COLUMN/COUNT DETAILS')
df_pre_standardization.info()

fn_display_header('DISPLAY FEW ROWS IN PRE-STANDADIZATION DATASET')
df_pre_standardization.head().style.set_properties(**{'text-align': 'left'})

--------------------------------------------------------------------------------
           PRE-STANDADIZATION COLUMN/COUNT DETAILS
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   code_description  694 non-null    object
 1   code_snippet      694 non-null    object
 2   import_line       62 non-null     object
 3   Category          694 non-null    object
 4   function          694 non-null    object
 5   origin_str        694 non-null    object
dtypes: object(6)
memory usage: 32.7+ KB
--------------------------------------------------------------------------------
           DISPLAY FEW ROWS IN PRE-STANDADIZATION DATASET
--------------------------------------------------------------------------------


,code_description,code_snippet,import_line,Category,function,origin_str
0,Pyspark code to Write a DataFrame into a CSV file.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,original
1,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,original
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'Hyukjin Kwon'.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,original
3,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,original
4,Pyspark code to Write a DataFrame into a JSON file.,"df.write.mode(""overwrite"").format(""json"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.format,original


In [17]:
# KEEP STANDARDIZATION DATASET WITH DESIRED COLUMNS ONLY. RENAME DERIVED COLUMNS

df_standard_comments = df_standardize_comments_3[
    (~df_standardize_comments_3['filter'].str.contains('IGNORE'))
    & (~df_standardize_comments_3['filter'].str.contains('=NA='))
    & (df_standardize_comments_3['code_desc_1'].str.contains('(LIT_(INT|DEC|STR)_\d+|COL_[A-Z]|YEAR|MONTH|DAY|HOUR|MIN|SEC)'))
].copy()

# Keep only necessary columns
df_standard_comments = df_standard_comments[['code_desc_1', 'code_snippet_1', 'import_line', 'Category', 'function']]

# Rename derived columns to original names
df_standard_comments.rename(columns={'code_desc_1': 'code_description', 'code_snippet_1': 'code_snippet'}, inplace=True)

# Add a column to identify the original records
df_standard_comments['origin_str'] = 'standardization'

fn_display_header('STANDARDIZATION DATASET COLUMN/COUNT DETAILS')
df_standard_comments.info()

fn_display_header('DISPLAY FEW ROWS IN PRE-STANDADIZATION DATASET')
df_standard_comments.head().style.set_properties(**{'text-align': 'left'})

--------------------------------------------------------------------------------
           STANDARDIZATION DATASET COLUMN/COUNT DETAILS
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 521 entries, 2 to 693
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   code_description  521 non-null    object
 1   code_snippet      521 non-null    object
 2   import_line       49 non-null     object
 3   Category          521 non-null    object
 4   function          521 non-null    object
 5   origin_str        521 non-null    object
dtypes: object(6)
memory usage: 28.5+ KB
--------------------------------------------------------------------------------
           DISPLAY FEW ROWS IN PRE-STANDADIZATION DATASET
--------------------------------------------------------------------------------


,code_description,code_snippet,import_line,Category,function,origin_str
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'LIT_STR_0'.The data is read from file or directory MY_DIR.,"MY_DF = spark.read.csv(MY_DIR, schema=MY_DF.schema, nullValue=""LIT_STR_0"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,standardization
14,"Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'LIT_STR_0', and 'header' option set to `True`.The data is read from file or directory MY_DIR.","MY_DF = spark.read.load(MY_DIR, schema=MY_DF.schema, format=""csv"", nullValue=""LIT_STR_0"", header=True)",nan,Input/Output,pyspark.sql.DataFrameReader.load,standardization
18,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'LIT_STR_0'.The data is read from file or directory MY_DIR.,"MY_DF = spark.read.schema(MY_DF.schema).option(""nullValue"", ""LIT_STR_0"").format('csv').load(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.option,standardization
22,"Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'LIT_STR_0', and 'header' option set to `True`.The data is read from file or directory MY_DIR.","MY_DF = spark.read.options(nullValue=""LIT_STR_0"",header=True).format('csv').load(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.options,standardization
40,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'LIT_STR_0'.The data is read from file or directory MY_DIR.,"MY_DF = spark.read.schema(MY_DF.schema).format(""csv"").option(""nullValue"", ""LIT_STR_0"").load(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameWriter.csv,standardization


In [18]:
 # CONCATENATE ORIGINAL AND STANDARDIZED DATASETS

# Concatenate
df_final = pd.concat([df_pre_standardization, df_standard_comments], ignore_index=True)

fn_display_header('STANDARDIZATION DATASET COLUMN/COUNT DETAILS')
df_final.info()

fn_display_header('DISPLAY FEW ROWS IN PRE-STANDADIZATION DATASET')
df_final.head().style.set_properties(**{'text-align': 'left'})

--------------------------------------------------------------------------------
           STANDARDIZATION DATASET COLUMN/COUNT DETAILS
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   code_description  1215 non-null   object
 1   code_snippet      1215 non-null   object
 2   import_line       111 non-null    object
 3   Category          1215 non-null   object
 4   function          1215 non-null   object
 5   origin_str        1215 non-null   object
dtypes: object(6)
memory usage: 57.1+ KB
--------------------------------------------------------------------------------
           DISPLAY FEW ROWS IN PRE-STANDADIZATION DATASET
--------------------------------------------------------------------------------


,code_description,code_snippet,import_line,Category,function,origin_str
0,Pyspark code to Write a DataFrame into a CSV file.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,original
1,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df.write.mode(""overwrite"").format(""csv"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.csv,original
2,Pyspark code to Read the CSV file as a DataFrame with 'nullValue' option set to 'Hyukjin Kwon'.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,original
3,Pyspark code which Loads a CSV file and returns the result as a DataFrame. This function will go through the input once to determine the input schema ifinferSchema is enabled. To avoid going through the entire data once disableinferSchema option or specify the schema explicitly using schema.,"df = spark.read.csv(MY_DIR, schema=df.schema, nullValue=""Hyukjin Kwon"")",nan,Input/Output,pyspark.sql.DataFrameReader.csv,original
4,Pyspark code to Write a DataFrame into a JSON file.,"df.write.mode(""overwrite"").format(""json"").save(MY_DIR)",nan,Input/Output,pyspark.sql.DataFrameReader.format,original


In [19]:
# WRITE FINAL DATASET TO CSV AND DOWNLOAD

DOWNLOAD_FLAG = 'N'
if DOWNLOAD_FLAG == 'Y':
  df_final.to_csv('ETL_P3_manual_data_standardization.csv')

  from google.colab import files
  files.download('ETL_P3_manual_data_standardization.csv')